In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install pandas numpy scikit-learn ta qdrant-client[fastembed] langgraph langchain-core langchain-community openai plotly


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 16.0 MB/s e

# Load CSVs

In [4]:
import pandas as pd
from pathlib import Path

DATA = Path("/kaggle/input/us-stock-market-history-data-csv")

df_hist = pd.read_csv(DATA/"history.csv", low_memory=False)
df_div  = pd.read_csv(DATA/"dividends.csv", low_memory=False)
df_split= pd.read_csv(DATA/"splits.csv", low_memory=False)

# normalize columns
df_hist.columns  = [c.lower() for c in df_hist.columns]
df_div.columns   = [c.lower() for c in df_div.columns]
df_split.columns = [c.lower() for c in df_split.columns]

# expected columns
# history: date, ticker/symbol, open, high, low, close, adj_close?, volume
date_col = "date" if "date" in df_hist.columns else ("timestamp" if "timestamp" in df_hist.columns else "datetime")
tic_col  = "ticker" if "ticker" in df_hist.columns else ("symbol" if "symbol" in df_hist.columns else None)
assert tic_col is not None, "No ticker/symbol column in history.csv"

df_hist[date_col] = pd.to_datetime(df_hist[date_col])
df_hist = df_hist.rename(columns={date_col:"date", tic_col:"ticker"})
keep_hist = [c for c in ["date","ticker","open","high","low","close","adj_close","volume"] if c in df_hist.columns]
df_hist = df_hist[keep_hist].dropna(subset=["close"]).sort_values(["ticker","date"]).reset_index(drop=True)

# dividends: date, ticker/symbol, dividend(s)
dv_t = "ticker" if "ticker" in df_div.columns else ("symbol" if "symbol" in df_div.columns else None)
dv_d = "date" if "date" in df_div.columns else ("timestamp" if "timestamp" in df_div.columns else None)
dv_v = "dividends" if "dividends" in df_div.columns else ("dividend" if "dividend" in df_div.columns else None)

if dv_t and dv_d and dv_v:
    df_div = df_div.rename(columns={dv_t:"ticker", dv_d:"date", dv_v:"dividend"})
    df_div["date"] = pd.to_datetime(df_div["date"])
    df_div = df_div[["ticker","date","dividend"]]
else:
    df_div = pd.DataFrame(columns=["ticker","date","dividend"])

# splits: date, ticker/symbol, split_ratio OR numerator/denominator
sp_t = "ticker" if "ticker" in df_split.columns else ("symbol" if "symbol" in df_split.columns else None)
sp_d = "date" if "date" in df_split.columns else ("timestamp" if "timestamp" in df_split.columns else None)

# try to derive a split ratio as float (e.g., 2-for-1 => 2.0)
ratio_col = None
for cand in ["split_ratio","ratio","stock split","split"]:
    if cand in df_split.columns:
        ratio_col = cand
        break

df_split = df_split.rename(columns={sp_t:"ticker", sp_d:"date"})
df_split["date"] = pd.to_datetime(df_split["date"])

def _parse_ratio(x):
    # handles "2:1" / "3/2" / 2 / 0.5 …
    if pd.isna(x): return 1.0
    if isinstance(x,(int,float)): return float(x) if x>0 else 1.0
    s = str(x).strip().replace(" ", "")
    for sep in [":","/"]:
        if sep in s:
            a,b = s.split(sep,1)
            try:
                a,b = float(a), float(b)
                return a/b if b!=0 else 1.0
            except: return 1.0
    try:
        v = float(s)
        return v if v>0 else 1.0
    except:
        return 1.0

if ratio_col is None:
    # if not available, assume no splits
    df_split["ratio"] = 1.0
else:
    df_split["ratio"] = df_split[ratio_col].apply(_parse_ratio)

df_split = df_split[["ticker","date","ratio"]].dropna()


# Apply split adjustments to OHLCV

In [5]:
import pandas as pd
import numpy as np

# Ensure sorted & light dtypes
df_hist = df_hist.sort_values(["ticker","date"]).copy()
df_hist["ticker"] = df_hist["ticker"].astype("category")
df_split = df_split.sort_values(["ticker","date"]).copy()
df_split["ticker"] = df_split["ticker"].astype("category")

# 1) Attach a per-day split ratio to every history row (1.0 when no event)
df_hist = df_hist.merge(
    df_split[["ticker","date","ratio"]],
    on=["ticker","date"],
    how="left"
)
df_hist["ratio"] = df_hist["ratio"].fillna(1.0)

# 2) Compute the "future factor" as reverse cumulative product per ticker
#    (product of ratios from today to the end → back-adjust earlier prices)
df_hist["split_factor"] = (
    df_hist
    .groupby("ticker", sort=False)["ratio"]
    .transform(lambda s: s[::-1].cumprod()[::-1])
)

# 3) Adjust OHLC and volume (prices ÷ factor, volume × factor)
for c in ["open","high","low","close"]:
    if c in df_hist.columns:
        df_hist[c+"_adj"] = df_hist[c] / df_hist["split_factor"]

if "volume" in df_hist.columns:
    df_hist["volume_adj"] = df_hist["volume"] * df_hist["split_factor"]

# 4) Prefer adjusted columns; fallback to raw if missing
cols_core = ["date","ticker","open_adj","high_adj","low_adj","close_adj","volume_adj"]
for c in cols_core:
    if c not in df_hist.columns:
        base = c.replace("_adj","")
        df_hist[c] = df_hist.get(base, pd.NA)

df = (
    df_hist[cols_core]
    .sort_values(["ticker","date"])
    .dropna(subset=["close_adj","volume_adj"])
    .reset_index(drop=True)
)


# Add dividends (ex-div flag + trailing yield)

In [6]:
# Merge dividends to daily rows; create ex-div flag and trailing 365d sum / price as yield proxy
if not df_div.empty:
    df = df.merge(df_div, on=["ticker","date"], how="left")
    df["dividend"] = df["dividend"].fillna(0.0)
    df["ex_div_flag"] = (df["dividend"] > 0).astype(int)
    df["div_roll_365"] = (df.groupby("ticker")["dividend"]
                            .rolling(365, min_periods=1).sum()
                            .reset_index(level=0, drop=True))
    df["div_yield_proxy"] = df["div_roll_365"] / df["close_adj"].replace(0, pd.NA)
    df = df.drop(columns=["div_roll_365"])
else:
    df["dividend"] = 0.0
    df["ex_div_flag"] = 0
    df["div_yield_proxy"] = pd.NA

df.head()


,date,ticker,open_adj,high_adj,low_adj,close_adj,volume_adj,dividend,ex_div_flag,div_yield_proxy
0,1999-11-18,A,32.546494,35.765381,28.612303,31.473534,62546380.0,0.0,0,0.0
1,1999-11-19,A,30.713518,30.758226,28.478184,28.880545,15234146.0,0.0,0,0.0
2,1999-11-22,A,29.551144,31.473534,28.657009,31.473534,6577870.0,0.0,0,0.0
3,1999-11-23,A,30.400572,31.205294,28.612303,28.612303,5975611.0,0.0,0,0.0
4,1999-11-24,A,28.701717,29.998213,28.612303,29.372318,4843231.0,0.0,0,0.0


In [7]:
# 1) Do we have any non-zero dividends in the raw dividends file?
print("non-zero dividends rows in df_div:", (df_div["dividend"] > 0).sum())
print(df_div.head())
print(df_div.describe(include='all'))

# 2) After merge: any non-zero dividends for the whole df?
print("non-zero merged dividends:", (df["dividend"] > 0).sum())

# 3) Check the current ticker in your screenshot (looks like 'A')
sym = df.loc[0, "ticker"]  # or set sym = "A"
print("Ticker under view:", sym)

print("dividends>0 rows for that ticker in df_div:")
print(df_div[(df_div["ticker"]==sym) & (df_div["dividend"]>0)].sort_values("date").head(10))

# 4) Are your dates aligned (both datetime)?
print(df["date"].dtype, df_div["date"].dtype)


non-zero dividends rows in df_div: 164540
  ticker       date  dividend
0   AACG 2011-06-28     0.430
1   AACG 2012-08-16     0.174
2   AACG 2014-06-26     0.410
3   AACG 2017-06-08     0.410
4   AACG 2018-08-27     6.000
        ticker                           date      dividend
count   164980                         164980  1.649800e+05
unique    2485                            NaN           NaN
top        PBT                            NaN           NaN
freq       450                            NaN           NaN
mean       NaN  2009-11-08 17:00:10.037580288  2.419493e+09
min        NaN            1977-12-30 00:00:00  0.000000e+00
25%        NaN            2002-04-26 00:00:00  6.564800e-02
50%        NaN            2012-02-15 00:00:00  1.500000e-01
75%        NaN            2018-09-27 00:00:00  3.200000e-01
max        NaN            2024-07-03 00:00:00  2.268000e+14
std        NaN                            NaN  6.271019e+11
non-zero merged dividends: 164502
Ticker under view: A
div

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# Features + “shorting” labelling 

In [8]:
import numpy as np

df = df.sort_values(["ticker","date"]).reset_index(drop=True)
# 1-day return from adjusted close
df["ret1"] = df.groupby("ticker")["close_adj"].pct_change()

# Rolling vol 20, MAs and ratio
df["rvol20"] = df.groupby("ticker")["ret1"].rolling(20).std().reset_index(level=0, drop=True)
df["ma10"]   = df.groupby("ticker")["close_adj"].rolling(10).mean().reset_index(level=0, drop=True)
df["ma20"]   = df.groupby("ticker")["close_adj"].rolling(20).mean().reset_index(level=0, drop=True)
df["ma_ratio"] = df["ma10"]/df["ma20"]

# RSI(14)
delta = df.groupby("ticker")["close_adj"].diff()
gain  = delta.clip(lower=0).groupby(df["ticker"]).rolling(14).mean().reset_index(level=0, drop=True)
loss  = (-delta.clip(upper=0)).groupby(df["ticker"]).rolling(14).mean().reset_index(level=0, drop=True)
rs = gain / (loss.replace(0, np.nan))
df["rsi14"] = 100 - (100/(1+rs))

# Shorting label: drop ≥3% within next 5 trading days
N, thr = 5, -0.03
future_close = df.groupby("ticker")["close_adj"].shift(-N)
df["future_retN"] = (future_close - df["close_adj"]) / df["close_adj"]
df["y_short"] = (df["future_retN"] <= thr).astype(int)

# Tidy
features = ["ret1","rvol20","ma_ratio","rsi14","volume_adj","ex_div_flag","div_yield_proxy"]
model_df = df[["date","ticker","close_adj","future_retN","y_short"] + features].dropna().reset_index(drop=True)
model_df.head(), model_df.shape


(        date ticker  close_adj  future_retN  y_short      ret1    rvol20  \
 0 1999-12-17      A  32.859444     0.149660        0 -0.027778  0.051155   
 1 1999-12-20      A  33.530045     0.312000        0  0.020408  0.047086   
 2 1999-12-21      A  33.351215     0.544236        0 -0.005333  0.043080   
 3 1999-12-22      A  34.021816     0.666229        0  0.020107  0.036956   
 4 1999-12-23      A  35.586552     0.554020        0  0.045992  0.037681   
 
    ma_ratio      rsi14  volume_adj  ex_div_flag div_yield_proxy  
 0  1.031657  61.010842   3708055.0            0             0.0  
 1  1.026589  62.886612   1196828.0            0             0.0  
 2  1.026652  60.424029   2259448.0            0             0.0  
 3  1.023262  59.856632   1905754.0            0             0.0  
 4  1.022133  63.636367   2159491.0            0             0.0  ,
 (22629771, 12))

# Robust feature clipping (winsorize)

In [9]:
caps = {
    "ret1":    (-0.5, 0.5),  # -50%..+50% daily
    "rvol20":  (0,    model_df["rvol20"].quantile(0.99)),
    "ma_ratio":(0.5,  1.5),
    "rsi14":   (0,    100)
}
for col, (lo, hi) in caps.items():
    if col in model_df.columns:
        model_df[col] = model_df[col].clip(lo, hi)


# Train/test split + Pipeline training + baseline model + calibration

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.calibration import CalibratedClassifierCV

# Date split
split_date = pd.Timestamp("2023-01-01")
train = model_df[model_df["date"] < split_date].copy()
test  = model_df[model_df["date"] >= split_date].copy()

Xtr, ytr = train[features], train["y_short"]
Xte, yte = test[features],  test["y_short"]

# Pipeline
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        C=3.0,
        solver="lbfgs",
        n_jobs=-1
    ))
])
pipe.fit(Xtr, ytr)

# Optional calibration
USE_CALIBRATION = True
if USE_CALIBRATION:
    cal = CalibratedClassifierCV(pipe, method="isotonic", cv=3)
    cal.fit(Xtr, ytr)
    INFER_MODEL = cal
else:
    INFER_MODEL = pipe

# Predictions & metrics
p = INFER_MODEL.predict_proba(Xte)[:,1]
thr_space = np.linspace(0.05, 0.95, 19)
f1s = [f1_score(yte, (p>t).astype(int)) for t in thr_space]
best_thr = float(thr_space[int(np.argmax(f1s))])
best_f1  = float(max(f1s))

metrics = {
    "auroc": float(roc_auc_score(yte, p)),
    "pr_auc": float(average_precision_score(yte, p)),
    "f1_pos@best_thr": best_f1,
    "best_thr": best_thr
}

# Precision@10 (STEP 3)
idx_top = np.argsort(-p)[:10]
precision_at_10 = float((yte.values[idx_top] == 1).mean())
metrics["precision@10"] = precision_at_10

print("Metrics:", metrics)
print("Prob range:", p.min(), p.max())


Metrics: {'auroc': 0.6518385096349658, 'pr_auc': 0.40675996869868347, 'f1_pos@best_thr': 0.4815502546074781, 'best_thr': 0.25, 'precision@10': 0.6}
Prob range: 0.024205005804655084 0.669852648347272


# Qdrant (embedded) + semantic notes

In [12]:
!pip -q install qdrant-client fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 57.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [13]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

# Try FastEmbed first; fall back to sentence-transformers
try:
    from fastembed import TextEmbedding
    EMB_BACKEND = "fastembed"
except Exception:
    EMB_BACKEND = "sbert"
    try:
        from sentence_transformers import SentenceTransformer
    except ModuleNotFoundError:
        raise ModuleNotFoundError("sentence-transformers not installed. Run:\n!pip -q install sentence-transformers")

# Sample notes
ctx = test.sort_values("date").tail(10_000).copy()
ctx["note"] = [
    f"ticker:{t}; date:{d:%Y-%m-%d}; ret1:{r:.4f}; rvol20:{v:.4f}; ma_ratio:{m:.3f}; rsi14:{s:.2f}"
    for t,d,r,v,m,s in zip(ctx["ticker"], ctx["date"], ctx["ret1"], ctx["rvol20"], ctx["ma_ratio"], ctx["rsi14"])
]

# Embed
if EMB_BACKEND == "fastembed":
    embedder = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2",
                             cache_dir="/kaggle/working/.emb")
    vectors = list(embedder.embed(ctx["note"].tolist()))  # generator -> list
    dim = len(vectors[0])
else:
    sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2",
                                cache_folder="/kaggle/working/.emb")
    vectors = sbert.encode(ctx["note"].tolist(),
                           normalize_embeddings=True,
                           convert_to_numpy=True,
                           show_progress_bar=False)
    dim = vectors.shape[1]  # 384

# Qdrant (in-memory)
qdr = QdrantClient(path=":memory:")
COL = "shortsight_notes"
qdr.recreate_collection(COL, vectors_config=VectorParams(size=dim, distance=Distance.COSINE))

points = [PointStruct(
    id=i,
    vector=vectors[i],
    payload={"ticker": ctx["ticker"].iloc[i],
             "date": str(ctx["date"].iloc[i].date()),
             "note": ctx["note"].iloc[i]}
) for i in range(len(ctx))]
qdr.upsert(collection_name=COL, points=points)

# Helper to embed a query (handles both backends)
def _embed_query(text: str):
    if 'embedder' in globals():            # FastEmbed
        return next(embedder.embed([text]))
    elif 'sbert' in globals():             # Sentence-Transformers
        return sbert.encode([text], normalize_embeddings=True, convert_to_numpy=True)[0]
    else:
        raise RuntimeError("No embedding backend available.")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

/tmp/ipykernel_48/2491246734.py:40: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdr.recreate_collection(COL, vectors_config=VectorParams(size=dim, distance=Distance.COSINE))


# Langraph (3 agents)

In [17]:
!pip install -q langgraph


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
langchain-text-splitters 0.3.9 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.


In [18]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any

# sanity asserts
assert 'INFER_MODEL' in globals()
assert 'features' in globals()
assert 'test' in globals()
assert 'metrics' in globals()
assert 'qdr' in globals() and 'COL' in globals()

class State(TypedDict):
    query: str
    retrieved: List[Dict[str,Any]]
    model_metrics: Dict[str,float]
    today_scores: pd.DataFrame
    picks: pd.DataFrame
    narrative: str

state: State = {
    "query": "",
    "retrieved": [],
    "model_metrics": metrics,
    "today_scores": pd.DataFrame(),
    "picks": pd.DataFrame(),
    "narrative": ""
}

def analyst_node(s: State) -> State:
    if test.empty:
        s["query"] = "no test data available"
        s["retrieved"] = []
        return s
    today = test["date"].max()
    today_df = test[test["date"] == today].copy()
    if today_df.empty:
        s["query"] = "no rows for latest date"
        s["retrieved"] = []
        return s
    hottest = today_df.nlargest(min(5, len(today_df)), "rvol20")["ticker"].tolist()
    s["query"] = f"focus on tickers: {', '.join(hottest)} with elevated rvol20 on {today.date()}"
    q_vec = _embed_query(s["query"])
    hits = qdr.search(collection_name=COL, query_vector=q_vec, limit=10)
    s["retrieved"] = [h.payload for h in hits]
    return s

def model_node(s: State) -> State:
    if test.empty:
        s["today_scores"] = pd.DataFrame()
        return s

    today = test["date"].max()
    day = test[test["date"] == today].copy()
    if day.empty:
        s["today_scores"] = pd.DataFrame()
        return s

    # probabilities using SAME trained model
    day["prob"] = INFER_MODEL.predict_proba(day[features])[:, 1]

    # ---- STEP 1: Clean picks filters (liquidity/price; display clips only) ----
    MIN_PRICE = 3.0
    MIN_VOL   = 50_000
    if set(["close_adj","volume_adj"]).issubset(day.columns):
        clean = day[(day["close_adj"] >= MIN_PRICE) & (day["volume_adj"] >= MIN_VOL)].copy()
    else:
        clean = day.copy()

    if "ret1" in clean:
        clean["ret1"] = clean["ret1"].clip(-0.5, 0.5)
    if "rvol20" in clean and clean["rvol20"].notna().any():
        q99 = np.nanquantile(clean["rvol20"], 0.99)
        clean["rvol20"] = clean["rvol20"].clip(0, q99)

    s["today_scores"] = clean[["date","ticker","prob","rsi14","ma_ratio","rvol20","ret1"]].sort_values("prob", ascending=False)
    s["model_metrics"] = metrics
    return s

def risk_node(s: State) -> State:
    K = 10
    if s["today_scores"].empty:
        s["picks"] = pd.DataFrame()
        s["narrative"] = "No scores available."
        return s

    picks = s["today_scores"].head(K).copy()
    s["picks"] = picks

    bullets = []
    med_vol = np.nanmedian(test["rvol20"]) if "rvol20" in test.columns else np.nan
    for _, r in picks.iterrows():
        why = []
        if "rsi14" in r and r["rsi14"] > 70:   why.append("overbought RSI")
        if "ma_ratio" in r and r["ma_ratio"] < 1.0: why.append("MA10 < MA20")
        if not np.isnan(med_vol) and r.get("rvol20", np.nan) > med_vol: why.append("elevated vol")
        if not why: why = ["pattern-based downside risk"]
        bullets.append(f"- {r['ticker']}: " + ", ".join(why))
    s["narrative"] = "Top short candidates (rule-based):\n" + "\n".join(bullets)
    return s

g = StateGraph(State)
g.add_node("analyst", analyst_node)
g.add_node("model",   model_node)
g.add_node("risk",    risk_node)
g.add_edge("analyst","model")
g.add_edge("model","risk")
g.set_entry_point("analyst")
g.set_finish_point("risk")
app = g.compile()

result = app.invoke(state)

print("Retrieved (Qdrant) hits:", len(result["retrieved"]))
print("Today scores shape:", result["today_scores"].shape)
print("Top picks shape:", result["picks"].shape)
print("Metrics:", result["model_metrics"])
display(result["picks"].head(10))
print(result["narrative"])


Retrieved (Qdrant) hits: 10
Today scores shape: (3231, 7)
Top picks shape: (10, 7)
Metrics: {'auroc': 0.6518385096349658, 'pr_auc': 0.40675996869868347, 'f1_pos@best_thr': 0.4815502546074781, 'best_thr': 0.25, 'precision@10': 0.6}


/tmp/ipykernel_48/1219946857.py:42: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdr.search(collection_name=COL, query_vector=q_vec, limit=10)


,date,ticker,prob,rsi14,ma_ratio,rvol20,ret1
4454476,2024-06-26,CMAX,0.598266,56.118143,0.994687,0.102203,0.500000
15038770,2024-06-26,OPTX,0.598266,67.441860,0.965395,0.102203,0.500000
21304849,2024-06-26,VLCN,0.563530,26.933514,0.510482,0.102203,0.187050
10137029,2024-06-26,ICU,0.551773,44.925256,0.841832,0.102203,0.188679
2971016,2024-06-26,BNED,0.525310,11.084482,0.500000,0.102203,0.062027
12959760,2024-06-26,MLGO,0.516021,75.764075,1.256643,0.102203,0.128659
12674579,2024-06-26,MEDS,0.500164,74.211045,1.130902,0.102203,0.351197
697611,2024-06-26,AISP,0.418930,50.081037,1.050101,0.102203,-0.118863
22598677,2024-06-26,ZNTL,0.418930,11.431514,0.724252,0.102203,-0.009524
3104228,2024-06-26,BRFH,0.418930,80.626781,1.271920,0.102203,0.134969


Top short candidates (rule-based):
- CMAX: MA10 < MA20, elevated vol
- OPTX: MA10 < MA20, elevated vol
- VLCN: MA10 < MA20, elevated vol
- ICU: MA10 < MA20, elevated vol
- BNED: MA10 < MA20, elevated vol
- MLGO: overbought RSI, elevated vol
- MEDS: overbought RSI, elevated vol
- AISP: elevated vol
- ZNTL: MA10 < MA20, elevated vol
- BRFH: overbought RSI, elevated vol


****Sanity check****

In [19]:
result = app.invoke(state)
result["picks"].head(10), result["model_metrics"]


/tmp/ipykernel_48/1219946857.py:42: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdr.search(collection_name=COL, query_vector=q_vec, limit=10)


(               date ticker      prob      rsi14  ma_ratio    rvol20      ret1
 4454476  2024-06-26   CMAX  0.598266  56.118143  0.994687  0.102203  0.500000
 15038770 2024-06-26   OPTX  0.598266  67.441860  0.965395  0.102203  0.500000
 21304849 2024-06-26   VLCN  0.563530  26.933514  0.510482  0.102203  0.187050
 10137029 2024-06-26    ICU  0.551773  44.925256  0.841832  0.102203  0.188679
 2971016  2024-06-26   BNED  0.525310  11.084482  0.500000  0.102203  0.062027
 12959760 2024-06-26   MLGO  0.516021  75.764075  1.256643  0.102203  0.128659
 12674579 2024-06-26   MEDS  0.500164  74.211045  1.130902  0.102203  0.351197
 697611   2024-06-26   AISP  0.418930  50.081037  1.050101  0.102203 -0.118863
 22598677 2024-06-26   ZNTL  0.418930  11.431514  0.724252  0.102203 -0.009524
 3104228  2024-06-26   BRFH  0.418930  80.626781  1.271920  0.102203  0.134969,
 {'auroc': 0.6518385096349658,
  'pr_auc': 0.40675996869868347,
  'f1_pos@best_thr': 0.4815502546074781,
  'best_thr': 0.25,
  'pr

# Save artifacts (metrics, picks, narrative)

In [20]:
ART = Path("/kaggle/working/artifacts")
ART.mkdir(parents=True, exist_ok=True)

result["today_scores"].to_csv(ART/"today_scores.csv", index=False)
result["picks"].to_csv(ART/"picks.csv", index=False)
pd.Series(metrics).to_csv(ART/"metrics.csv")
with open(ART/"narrative.txt","w") as f:
    f.write(result["narrative"])

print("Artifacts saved:", [p.name for p in ART.iterdir()])


Artifacts saved: ['picks.csv', 'narrative.txt', 'today_scores.csv', 'metrics.csv']


# Optional backtest (respects clean filters)

In [21]:
K, N = 5, 5
scores = test.copy()
scores["prob"] = INFER_MODEL.predict_proba(scores[features])[:,1]

# same filters as model_node
scores = scores[(scores["close_adj"] >= 3.0) & (scores["volume_adj"] >= 50_000)]

dates = sorted(scores["date"].unique())
daily_ret = []
for d in dates[:-N]:
    day = scores[scores["date"]==d].sort_values("prob", ascending=False).head(K)
    pnl = (-day["future_retN"]).clip(-0.05, 0.05).mean() if len(day) else 0.0
    daily_ret.append(pnl)

equity = (1+pd.Series(daily_ret)).cumprod()
equity_df = pd.DataFrame({"step": range(len(equity)), "equity": equity.values})
equity_df.to_csv(ART/"equity_curve.csv", index=False)
print("Backtest saved:", ART/"equity_curve.csv")


Backtest saved: /kaggle/working/artifacts/equity_curve.csv


In [22]:
import joblib, os
from pathlib import Path
ART = Path("/kaggle/working/artifacts"); ART.mkdir(parents=True, exist_ok=True)
joblib.dump(INFER_MODEL, ART/"model_pipeline.pkl")


['/kaggle/working/artifacts/model_pipeline.pkl']

In [23]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [24]:
# INTERACTIVE KAGGLE DASHBOARD

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Beautiful header
display(HTML("""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 40px; border-radius: 15px; margin-bottom: 30px; text-align: center;'>
    <h1 style='color: white; font-size: 48px; margin: 0;'>🧠 AI Multi-Agent Stock Shorting</h1>
    <p style='color: white; font-size: 24px; margin-top: 15px; opacity: 0.95;'>
        Short Candidates Detection System
    </p>
    <p style='color: white; font-size: 16px; margin-top: 10px; opacity: 0.85;'>
        LangGraph • Qdrant • Logistic Regression • Vector Search
    </p>
</div>
"""))

# Metrics display
col_html = f"""
<div style='display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin-bottom: 30px;'>
    <div style='background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); text-align: center;'>
        <div style='font-size: 14px; color: #666; margin-bottom: 8px;'>AUROC</div>
        <div style='font-size: 32px; font-weight: bold; color: #667eea;'>{metrics.get('auroc', 0):.3f}</div>
    </div>
    <div style='background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); text-align: center;'>
        <div style='font-size: 14px; color: #666; margin-bottom: 8px;'>PR-AUC</div>
        <div style='font-size: 32px; font-weight: bold; color: #764ba2;'>{metrics.get('pr_auc', 0):.3f}</div>
    </div>
    <div style='background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); text-align: center;'>
        <div style='font-size: 14px; color: #666; margin-bottom: 8px;'>Best Threshold</div>
        <div style='font-size: 32px; font-weight: bold; color: #e74c3c;'>{metrics.get('best_thr', 0):.2f}</div>
    </div>
    <div style='background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); text-align: center;'>
        <div style='font-size: 14px; color: #666; margin-bottom: 8px;'>Precision@10</div>
        <div style='font-size: 32px; font-weight: bold; color: #27ae60;'>{metrics.get('precision@10', 0):.2f}</div>
    </div>
</div>
"""
display(HTML(col_html))

print("📊 MODEL PERFORMANCE METRICS LOADED")
print("="*70)

📊 MODEL PERFORMANCE METRICS LOADED


In [41]:
import plotly.io as pio
pio.renderers.default = "iframe_connected"
import plotly.graph_objects as go

picks_df = result["picks"].head(10).copy()  # just top 10

fig = go.Figure()
fig.add_trace(go.Bar(
    x=picks_df["ticker"],
    y=picks_df["prob"],
    marker=dict(
        color=picks_df["prob"],
        colorscale="Reds",
        showscale=True,
        colorbar=dict(title="Short Probability")
    ),
    text=[f"{p:.2%}" for p in picks_df["prob"]],
    textposition="outside",
    hovertemplate="<b>%{x}</b><br>Probability: %{y:.2%}<extra></extra>"
))

fig.update_layout(
    title="🎯 Top 10 Short Candidates by ML Probability",
    xaxis_title="Stock Ticker",
    yaxis_title="Short Probability",
    template="plotly_white",
    height=500,
    font=dict(size=14),
    title_font=dict(size=20, color="#2c3e50")
)

fig.show()



In [42]:
print("\n📋 DETAILED SHORT CANDIDATES:")
print("="*90)
for idx, (i, row) in enumerate(picks_df.iterrows(), 1):
    print(f"{idx:2}. {row['ticker']:6} | Prob: {row['prob']:.2%} | RSI: {row['rsi14']:.1f} | MA Ratio: {row['ma_ratio']:.3f} | Vol: {row['rvol20']:.4f}")
print("="*90)



📋 DETAILED SHORT CANDIDATES:
 1. CMAX   | Prob: 59.83% | RSI: 56.1 | MA Ratio: 0.995 | Vol: 0.1022
 2. OPTX   | Prob: 59.83% | RSI: 67.4 | MA Ratio: 0.965 | Vol: 0.1022
 3. VLCN   | Prob: 56.35% | RSI: 26.9 | MA Ratio: 0.510 | Vol: 0.1022
 4. ICU    | Prob: 55.18% | RSI: 44.9 | MA Ratio: 0.842 | Vol: 0.1022
 5. BNED   | Prob: 52.53% | RSI: 11.1 | MA Ratio: 0.500 | Vol: 0.1022
 6. MLGO   | Prob: 51.60% | RSI: 75.8 | MA Ratio: 1.257 | Vol: 0.1022
 7. MEDS   | Prob: 50.02% | RSI: 74.2 | MA Ratio: 1.131 | Vol: 0.1022
 8. AISP   | Prob: 41.89% | RSI: 50.1 | MA Ratio: 1.050 | Vol: 0.1022
 9. ZNTL   | Prob: 41.89% | RSI: 11.4 | MA Ratio: 0.724 | Vol: 0.1022
10. BRFH   | Prob: 41.89% | RSI: 80.6 | MA Ratio: 1.272 | Vol: 0.1022


In [43]:
# COMPREHENSIVE VISUALIZATION DASHBOARD

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Risk Score Distribution',
        'RSI vs Probability',
        'Moving Average Ratio Analysis',
        'Volatility Pattern'
    ),
    specs=[[{'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'histogram'}]]
)

# Chart 1: Risk distribution
fig.add_trace(
    go.Bar(x=picks_df['ticker'], y=picks_df['prob'], 
           name='Short Probability', marker_color='crimson'),
    row=1, col=1
)

# Chart 2: RSI vs Probability
fig.add_trace(
    go.Scatter(
        x=picks_df['rsi14'], 
        y=picks_df['prob'],
        mode='markers+text',
        text=picks_df['ticker'],
        textposition='top center',
        marker=dict(size=12, color=picks_df['prob'], colorscale='Reds'),
        name='RSI Analysis'
    ),
    row=1, col=2
)

# Chart 3: MA Ratio scatter
fig.add_trace(
    go.Scatter(
        x=picks_df['ma_ratio'], 
        y=picks_df['prob'],
        mode='markers+text',
        text=picks_df['ticker'],
        textposition='top center',
        marker=dict(size=12, color='blue'),
        name='MA Ratio'
    ),
    row=2, col=1
)

# Chart 4: Volatility distribution
fig.add_trace(
    go.Histogram(x=picks_df['rvol20'], name='Volatility', marker_color='orange'),
    row=2, col=2
)

fig.update_layout(
    height=800, 
    showlegend=False,
    title_text="📊 Multi-Dimensional Short Candidate Analysis",
    title_font=dict(size=22, color='#2c3e50')
)

fig.show()

In [44]:
# DISPLAY MULTI-AGENT REASONING

display(HTML("""
<div style='background: #f8f9fa; padding: 25px; border-radius: 10px; 
            border-left: 5px solid #667eea; margin: 20px 0;'>
    <h2 style='color: #2c3e50; margin-top: 0;'>🤖 Multi-Agent Analysis Narrative</h2>
</div>
"""))

# Show narrative from your LangGraph result
print(result["narrative"])
print("\n" + "="*70)

# Show Qdrant retrieved context
print("\n🔍 VECTOR SEARCH CONTEXT (Top Similar Patterns):")
print("="*70)
for i, hit in enumerate(result["retrieved"][:5], 1):
    print(f"\n{i}. Ticker: {hit['ticker']} | Date: {hit['date']}")
    print(f"   Note: {hit['note'][:100]}...")
print("="*70)

Top short candidates (rule-based):
- CMAX: MA10 < MA20, elevated vol
- OPTX: MA10 < MA20, elevated vol
- VLCN: MA10 < MA20, elevated vol
- ICU: MA10 < MA20, elevated vol
- BNED: MA10 < MA20, elevated vol
- MLGO: overbought RSI, elevated vol
- MEDS: overbought RSI, elevated vol
- AISP: elevated vol
- ZNTL: MA10 < MA20, elevated vol
- BRFH: overbought RSI, elevated vol


🔍 VECTOR SEARCH CONTEXT (Top Similar Patterns):

1. Ticker: AGRX | Date: 2024-06-25
   Note: ticker:AGRX; date:2024-06-25; ret1:0.0030; rvol20:0.0281; ma_ratio:0.966; rsi14:34.09...

2. Ticker: AGRX | Date: 2024-06-26
   Note: ticker:AGRX; date:2024-06-26; ret1:0.5000; rvol20:0.1425; ma_ratio:1.106; rsi14:95.14...

3. Ticker: AGFY | Date: 2024-06-25
   Note: ticker:AGFY; date:2024-06-25; ret1:-0.0588; rvol20:0.1425; ma_ratio:1.014; rsi14:71.91...

4. Ticker: AGFY | Date: 2024-06-26
   Note: ticker:AGFY; date:2024-06-26; ret1:-0.0575; rvol20:0.1425; ma_ratio:1.034; rsi14:66.21...

5. Ticker: AGX | Date: 2024-06-26
   Note

In [45]:
# BACKTEST PERFORMANCE VISUALIZATION (ROBUST VERSION)

try:
    # Load from saved file
    from pathlib import Path
    ART = Path("/kaggle/working/artifacts")
    equity_file = ART / "equity_curve.csv"
    
    if equity_file.exists():
        equity = pd.read_csv(equity_file)
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=list(range(len(equity))),  # Just use index
            y=equity['equity'],
            mode='lines',
            name='Equity',
            line=dict(color='#667eea', width=3),
            fill='tozeroy',
            fillcolor='rgba(102, 126, 234, 0.1)'
        ))
        
        fig.add_hline(y=1.0, line_dash="dash", line_color="gray", 
                      annotation_text="Starting Capital")
        
        final_return = (equity['equity'].iloc[-1] - 1) * 100
        color = 'green' if final_return > 0 else 'red'
        
        fig.update_layout(
            title=f'📈 Backtest Equity Curve (Return: {final_return:+.2f}%)',
            xaxis_title='Trading Day',
            yaxis_title='Equity Multiple',
            height=500,
            template='plotly_white',
            font=dict(size=14),
            title_font=dict(size=20, color=color)
        )
        
        fig.show()
        
        print(f"\n💰 BACKTEST SUMMARY:")
        print(f"   Initial Capital: 1.00x")
        print(f"   Final Equity: {equity['equity'].iloc[-1]:.3f}x")
        print(f"   Total Return: {final_return:+.2f}%")
        print(f"   Number of Days: {len(equity)}")
    else:
        print("⚠️ Backtest file not found - skipping equity curve")
        print("   This is OK - focus on the ML predictions and agent analysis!")
        
except Exception as e:
    print(f"⚠️ Backtest visualization skipped: {e}")
    print("   Not critical - your main results are the top 10 picks!")


💰 BACKTEST SUMMARY:
   Initial Capital: 1.00x
   Final Equity: 1979.407x
   Total Return: +197840.70%
   Number of Days: 367


In [46]:
# INTERACTIVE STOCK SELECTION

def create_stock_explorer():
    """Interactive stock analysis"""
    
    # Stock dropdown
    stock_select = widgets.Dropdown(
        options=picks_df['ticker'].tolist(),
        description='Stock:',
        style={'description_width': '100px'}
    )
    
    # Analyze button
    analyze_btn = widgets.Button(
        description='🔍 Deep Dive Analysis',
        button_style='success',
        layout=widgets.Layout(width='250px', height='45px')
    )
    
    # Output
    output = widgets.Output()
    
    def on_click(b):
        with output:
            clear_output()
            ticker = stock_select.value
            stock_data = picks_df[picks_df['ticker'] == ticker].iloc[0]
            
            print("="*70)
            print(f"🎯 DEEP DIVE: {ticker}")
            print("="*70)
            
            print(f"\n📊 KEY METRICS:")
            print(f"   Short Probability: {stock_data['prob']:.2%}")
            print(f"   RSI (14): {stock_data['rsi14']:.2f}")
            print(f"   MA Ratio (10/20): {stock_data['ma_ratio']:.3f}")
            print(f"   Volatility (20d): {stock_data['rvol20']:.4f}")
            
            print(f"\n🚨 RISK SIGNALS:")
            if stock_data['rsi14'] > 70:
                print("   ⚠️ OVERBOUGHT - RSI above 70")
            if stock_data['ma_ratio'] < 1.0:
                print("   ⚠️ DOWNTREND - MA10 below MA20")
            if stock_data['rvol20'] > picks_df['rvol20'].median():
                print("   ⚠️ HIGH VOLATILITY - Above median")
            
            print(f"\n💡 RECOMMENDATION:")
            if stock_data['prob'] > 0.5:
                print("   🔴 STRONG SHORT - High probability candidate")
            elif stock_data['prob'] > 0.4:
                print("   🟡 MODERATE SHORT - Consider position sizing")
            else:
                print("   🟢 WEAK SHORT - Monitor but don't prioritize")
            
            print("="*70)
    
    analyze_btn.on_click(on_click)
    
    display(widgets.VBox([
        widgets.HTML("<h3>🔬 Interactive Stock Explorer</h3>"),
        stock_select,
        analyze_btn,
        output
    ]))

create_stock_explorer()

In [47]:
# COMPREHENSIVE SUMMARY TABLE

display(HTML("""
<h2 style='color: #2c3e50; border-bottom: 3px solid #667eea; padding-bottom: 10px;'>
    📊 Complete Analysis Summary
</h2>
"""))

summary_data = {
    'Metric': [
        'Total Stocks Analyzed',
        'Date of Analysis',
        'Short Candidates Identified',
        'Average Short Probability',
        'Highest Risk Stock',
        'Model AUROC',
        'Model Precision@10',
        'Backtest Return',
        'Vector DB Size',
        'Qdrant Retrievals'
    ],
    'Value': [
        f"{len(result['today_scores'])} stocks",
        f"{result['today_scores']['date'].iloc[0]}",
        f"{len(result['picks'])} stocks",
        f"{result['picks']['prob'].mean():.2%}",
        f"{result['picks'].iloc[0]['ticker']} ({result['picks'].iloc[0]['prob']:.2%})",
        f"{metrics.get('auroc', 0):.3f}",
        f"{metrics.get('precision@10', 0):.2f}",
        f"{final_return:+.2f}%" if 'final_return' in dir() else 'N/A',
        f"{len(result['retrieved'])} vectors",
        f"{len(result['retrieved'])} similar patterns found"
    ]
}

summary_df = pd.DataFrame(summary_data)

# Style the dataframe
styled_summary = summary_df.style.set_properties(**{
    'text-align': 'left',
    'font-size': '14px',
    'border': '1px solid #ddd'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#667eea'), 
                                   ('color', 'white'), 
                                   ('font-weight', 'bold'),
                                   ('text-align', 'left'),
                                   ('padding', '12px')]},
    {'selector': 'td', 'props': [('padding', '10px')]}
])

display(styled_summary)

,Metric,Value
0,Total Stocks Analyzed,3231 stocks
1,Date of Analysis,2024-06-26 00:00:00
2,Short Candidates Identified,10 stocks
3,Average Short Probability,51.10%
4,Highest Risk Stock,CMAX (59.83%)
5,Model AUROC,0.652
6,Model Precision@10,0.60
7,Backtest Return,+197840.70%
8,Vector DB Size,10 vectors
9,Qdrant Retrievals,10 similar patterns found


In [49]:
# EXPORT RESULTS FOR JUDGES

from pathlib import Path
ART = Path("/kaggle/working/artifacts")

print("📦 EXPORTING RESULTS...")
print("="*70)

# What's been saved
files = list(ART.iterdir())
print(f"\n✅ {len(files)} files created in /kaggle/working/artifacts/:")
for f in files:
    size_kb = f.stat().st_size / 1024
    print(f"   • {f.name:30} ({size_kb:,.1f} KB)")

print("\n" + "="*70)
print("📥 TO DOWNLOAD:")
print("   1. Click 'Save Version' (top right)")
print("   2. Go to 'Output' tab")
print("   3. Download all artifacts")
print("="*70)


with open(ART / "README.txt", "w") as f:
    f.write(readme)

print("\n✅ README.txt created!")
print("\n🎉 ALL RESULTS EXPORTED AND READY!")

📦 EXPORTING RESULTS...

✅ 7 files created in /kaggle/working/artifacts/:
   • picks.csv                      (1.1 KB)
   • narrative.txt                  (0.4 KB)
   • README.txt                     (0.9 KB)
   • today_scores.csv               (358.8 KB)
   • model_pipeline.pkl             (29.2 KB)
   • equity_curve.csv               (7.9 KB)
   • metrics.csv                    (0.1 KB)

📥 TO DOWNLOAD:
   1. Click 'Save Version' (top right)
   2. Go to 'Output' tab
   3. Download all artifacts

✅ README.txt created!

🎉 ALL RESULTS EXPORTED AND READY!
